In [2]:
import numpy as np
import sys
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris

In [3]:
# From ffnn.ipynb
def countSSError(results, expected_results):
    results = np.array(results)
    expected_results = np.array(expected_results)
    if results.shape != expected_results.shape:
        print("Error: array shape mismatch")
        sys.exit(1)
    else:
        return np.sum(np.square(results - expected_results))

Sklearn Using Iris Dataset

In [4]:
iris = load_iris()

learning_data = np.array(iris.data)
learning_target = np.array(iris.target)

classifier = MLPClassifier(
    hidden_layer_sizes=(5,3,3), 
    activation='relu', 
    max_iter=1000, 
    learning_rate='constant', 
    learning_rate_init=0.001, 
    random_state=1
    )

classifier.fit(learning_data, learning_target)
y_pred = classifier.predict(learning_data)

print("SKLearn MLPClassifier with 5,3,3,2 hidden layers and relu activation function")
print(f"Prediction:\n{y_pred}")
print()
print(f"Target:\n{learning_target}")
print()

weight_data = classifier.coefs_
bias_data = classifier.intercepts_

print("layer count: {}".format(classifier.n_layers_))
print("iteration count: {}".format(classifier.n_iter_))
print("output count: {}".format(classifier.n_outputs_))

print("Weight Input: \n", weight_data[0])
print("Bias Input: \n", bias_data[0])
print("Weight Output: \n", weight_data[2])
print("Bias Output: \n", bias_data[2])

error = countSSError(y_pred, learning_target)
print(f"Error: {error / len(learning_data) * 100} %")

SKLearn MLPClassifier with 5,3,3,2 hidden layers and relu activation function
Prediction:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]

Target:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]

Jumlah layer: 5
Jumlah iterasi: 1000
Jumlah fitur: 4
Jumlah output: 3
Weight Input: 
 [[-2.11237097e-02  3.13107969e-01 -1.55344599e-01 -4.62181600e-01
   2.39886078e-01]
 [-2.71512518e-01 -8.26239061e-01 -1.39626167e-04 -3.79599188e-01
   3.28595945e-01]
 [-2.10151629e-02  7.62541414e-0

c:\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Tensorflow Using Test Cases

In [316]:
import json
import tensorflow as tf

# Define the loss function
def loss_function(y_true, y_pred, activation_function):
    squared_error = tf.square(y_true - y_pred)
    loss = tf.reduce_sum(squared_error) / tf.cast(tf.shape(y_true)[0], tf.float32)
    return 0.5 * loss

# Load the JSON data
with open('testcase/mlp.json') as f:
    data = json.load(f)

In [317]:
# Define the model architecture
class TensorModel(tf.keras.Model):
    def __init__(self, input_size, layers):
        super(TensorModel, self).__init__()
        self.dense_layers = []
        for i, layer in enumerate(layers):
            activation_function = layer["activation_function"]
            num_neurons = layer["number_of_neurons"]
            self.dense_layers.append(tf.keras.layers.Dense(num_neurons, activation=activation_function, use_bias=False, name=f'dense_{i}'))
        self(tf.keras.Input(shape=(input_size,)))

    def call(self, inputs):
        x = inputs
        for layer in self.dense_layers:
            # Concatenate a column vector of ones to the input data
            x_with_bias = tf.concat([tf.ones((tf.shape(x)[0], 1)), x], axis=1)
            x = layer(x_with_bias)
        return x

# Parse JSON data
input_size = data["case"]["model"]["input_size"] 
layers = data["case"]["model"]["layers"]

# Extract input data from JSON and add bias
json_input = data["case"]["input"]
input_with_bias = json_input
input_data = tf.constant(input_with_bias, dtype=tf.float32)

# Extract target data from JSON and reshape it
json_target = data["case"]["target"]
target_data = tf.constant(json_target, dtype=tf.float32)
height, width = target_data.shape
target_data = tf.reshape(target_data, [1, height, width])

# Extract other parameters from JSON
initial_weights = [tf.constant(weights, dtype=tf.float32) for weights in data["case"]["initial_weights"]]
learning_rate = data["case"]["learning_parameters"]["learning_rate"]
batch_size = data["case"]["learning_parameters"]["batch_size"]
max_iteration = data["case"]["learning_parameters"]["max_iteration"]
error_threshold = data["case"]["learning_parameters"]["error_threshold"]

# Initialize the model
model = TensorModel(input_size, layers)

# Set the initial weights for the dense layers
for i, weights in enumerate(initial_weights):
    model.dense_layers[i].set_weights([weights])

optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

# Training
for iteration in range(max_iteration):
    total_loss = 0
    for i in range(0, input_data.shape[0], batch_size):
        x_batch = input_data[i:i+batch_size]
        y_batch = target_data[i:i+batch_size]

        with tf.GradientTape() as tape:
            predictions = model(x_batch, training=True)
            # activation_function = layers[i]["activation_function"]
            loss = loss_function(y_batch, predictions, activation_function)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        total_loss += loss
        
    avg_loss = total_loss / (input_data.shape[0] / batch_size) 

    # Check if the average loss is below the error threshold
    if avg_loss < error_threshold:
        print(f"Error threshold reached. Stopping training at iteration {iteration + 1}.")
        break

# Print the final weights and check
final_weights = [layer.get_weights()[0] for layer in model.dense_layers]
expected_final_weights = data["expect"]["final_weights"]

print("Final Weights:")
for i, weights in enumerate(final_weights):
    print(f"Layer {i+1} weights:")
    print(weights)
print("Stopped by:", "max_iteration" if iteration == max_iteration - 1 else "error_threshold")


Final Weights:
Layer 1 weights:
[[ 0.08592001  0.32276   ]
 [-0.33872002  0.46172   ]
 [ 0.449984    0.440072  ]]
Layer 2 weights:
[[ 0.2748    0.188   ]
 [ 0.435904 -0.53168 ]
 [ 0.68504   0.7824  ]]
Stopped by: max_iteration
